In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# --- Settings ---
model_path = "LandSlides.pt"   # Your trained landslide segmentation model
image_path = "l1.jpeg"         # Test image

# Load model & image
model = YOLO(model_path)
img = cv2.imread(image_path)
height, width = img.shape[:2]

# Run segmentation
results = model(img)[0]

if results.masks is None:
    print("No landslide detected.")
else:
    masks = results.masks.data.cpu().numpy()

    for mask in masks:
        # Resize mask to match original image shape
        mask = cv2.resize(mask, (width, height))
        mask = (mask > 0.5).astype(np.uint8)  # 1 = landslide, 0 = background

        # ✅ Find precise contour of landslide shape
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # ✅ Draw only outline/border of landslide (no fill)
        cv2.drawContours(img, contours, -1, (0, 0, 255), 2)  # Red outline

# ✅ Display final image with only border
cv2.namedWindow("Landslide Segmentation (Border Only)", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Landslide Segmentation (Border Only)", 1000, 700)
cv2.imshow("Landslide Segmentation (Border Only)", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
